In [1]:
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Define paths
dataset_path = "./dataset/train"

def load_data(dataset_path):
    data = []
    labels = []

    for category in os.listdir(dataset_path):  # car, bus, truck
        category_path = os.path.join(dataset_path, category)
        if os.path.isdir(category_path):
            for img_name in os.listdir(category_path):
                img_path = os.path.join(category_path, img_name)
                # Read and preprocess the image
                img = cv2.imread(img_path)
                img = cv2.resize(img, (128, 128))  # Resize for uniformity
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                edges = cv2.Canny(gray, 100, 200)  # Apply Canny edge detection

                # Flatten the edge-detected image as features
                data.append(edges.flatten())
                labels.append(category)

    return np.array(data), np.array(labels)

# Load the dataset
X, y = load_data(dataset_path)

# Encode labels (car -> 0, bus -> 1, truck -> 2)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train an SVM classifier
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred = svm_model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Prediction function
def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (128, 128))  # Resize for uniformity
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)  # Apply Canny edge detection
    features = edges.flatten().reshape(1, -1)  # Reshape for prediction

    # Predict the class and confidence
    predicted_class_index = svm_model.predict(features)[0]
    confidence = np.max(svm_model.predict_proba(features))
    predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]

    return predicted_label, confidence

Classification Report:
              precision    recall  f1-score   support

       buses       0.75      0.96      0.84        46
        cars       0.62      0.38      0.48        13
      trucks       0.50      0.10      0.17        10

    accuracy                           0.72        69
   macro avg       0.62      0.48      0.49        69
weighted avg       0.69      0.72      0.67        69

The predicted label for the test image is: cars
Confidence level: 0.56


In [6]:
# Test with a new image
test_image_path = "./test/truck2.jpg"  # Replace with your test image path
predicted_label, confidence = predict_image(test_image_path)
print(f"The predicted label for the test image is: {predicted_label}")
print(f"Confidence level: {confidence:.2f}")

The predicted label for the test image is: buses
Confidence level: 0.90
